In [2]:
import gc
gc.collect()

58

In [3]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import spacy
import en_core_web_sm
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
pd.options.display.max_colwidth = 100
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [4]:
# load data
data = pd.read_csv('/home/inblr01-30/Desktop/twcs/twcs.csv')

In [5]:
# filtering for customer tweets only
customers = data.loc[data['inbound']==True]

In [22]:
# running algos on a subset data due to computational restrictions
data_clean = customers.head(50000).copy()

In [7]:
data_clean.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messages and no one is responding as usual,1,4.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your connectivity for me and my whole family ? 🤥 💯,"11,13,14",15.0


In [8]:
# cleaning the text (removing tweeter handles, urls, extra spaces, punctuations, etc)
data_clean['text'] = data_clean['text'].str.replace('@[\w]*|http\S+|[,.;@#?!&$]+\ *',' ')
data_clean['text'] = data_clean['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data_clean['text'] = data_clean['text'].str.lower()
stop = stopwords.words('english')
data_clean['text'] = data_clean['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [9]:
# removing data points with missing texts
data_clean = data_clean.drop(['inbound','created_at','response_tweet_id','in_response_to_tweet_id'],axis=1)
data_clean['text'].replace('', np.nan, inplace=True)
data_clean.dropna(subset=['text'], inplace=True)

In [10]:
# splitting data into train and test
X_train, X_test = train_test_split(data_clean, test_size=0.2)
print (X_train.shape)
print (X_test.shape)

(39028, 4)
(9757, 4)


In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  # deacc=True removes punctuations

data_words = X_train.text.values.tolist()
data_words = list(sent_to_words(data_words))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [275]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)
#print(data_words_bigrams)

In [276]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp = en_core_web_sm.load()

In [277]:
# Do lemmatization keeping only noun, adj, vb, adv
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized = data_words_bigrams
#data_lemmatized

In [278]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus)

In [279]:
from datetime import datetime
datetime.now()

datetime.datetime(2020, 8, 30, 15, 56, 29, 547546)

In [280]:
# Build LDA model
"""
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       workers=3)
"""                            

'\nlda_model = gensim.models.LdaMulticore(corpus=corpus,\n                                       id2word=id2word,\n                                       num_topics=10, \n                                       random_state=100,\n                                       chunksize=100,\n                                       passes=10,\n                                       per_word_topics=True,\n                                       workers=3)\n'

In [281]:
datetime.now()

datetime.datetime(2020, 8, 30, 15, 56, 29, 561993)

In [282]:
"""
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
"""

'\n# Print the Keyword in the 10 topics\n#pprint(lda_model.print_topics())\ndoc_lda = lda_model[corpus]\n'

In [283]:
"""
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)
"""

"\n# Compute Coherence Score\ncoherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')\ncoherence_lda = coherence_model_lda.get_coherence()\nprint('Coherence Score: ', coherence_lda)\n"

In [284]:
# supporting function
def compute_coherence_values(corpus, dictionary, k):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta=0.91)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [285]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 15
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Validation sets
num_of_docs = len(corpus)
num_of_docs

39028

In [286]:

corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm(total=(len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word,k=k)
            # Save the model results
            model_results['Validation_Set'].append(corpus_title[i])
            model_results['Topics'].append(k)
            model_results['Coherence'].append(cv)
                    
            pbar.update(1)
    pd.DataFrame(model_results).to_csv('/home/inblr01-30/Desktop/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 13/13 [24:32<00:00, 113.25s/it]


In [287]:
#model_results['Coherence']

In [288]:
model_results_matrix = model_results.copy()
del model_results_matrix['Validation_Set']
model_results_matrix1 = pd.DataFrame(model_results_matrix)
#model_results_matrix1.head(20)

In [289]:
m = model_results_matrix1[model_results_matrix1['Coherence'] == model_results_matrix1.groupby('Topics')['Coherence'].transform('max')]
m

,Topics,Coherence
0,2,0.323461
1,3,0.392017
2,4,0.466704
3,5,0.471210
4,6,0.504031
5,7,0.544058
6,8,0.551519
7,9,0.511215
8,10,0.551590
9,11,0.576080


In [14]:
test_words = X_test.text.values.tolist()
test_words = list(sent_to_words(test_words))

In [16]:
# Build the bigram and trigram models
bigram_test = gensim.models.Phrases(test_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_test = gensim.models.Phrases(bigram_test[test_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_test = gensim.models.phrases.Phraser(bigram_test)
trigram_mod_test = gensim.models.phrases.Phraser(trigram_test)

In [18]:
test_words_bigrams = [bigram_mod_test[doc] for doc in test_words]

In [19]:
# Create Dictionary
id2word = corpora.Dictionary(test_words_bigrams)

# Create Corpus
texts = test_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [20]:
# Build LDA model

lda_model_test = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=11, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       workers=3)


In [21]:
doc_lda_test = lda_model_test[corpus]
coherence_model_lda_test = CoherenceModel(model=lda_model_test, texts=test_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda_test = coherence_model_lda_test.get_coherence()
print('Coherence Score: ', coherence_lda_test)

Coherence Score:  0.33625184673898567
